In [2]:
import random
import numpy as np
import operator
import math
import pandas as pd
from pandas import DataFrame


t = np.arange(0.01,10,0.01)

g = pd.read_csv('ha_d.csv')
x = g['x']
y = g['y']
dx = g['dx']
dy = g['dy']

f = {'r' : [], 'theta' : []}
for i in range(len(t)):
    f['r'].append(np.sqrt(x[i]**2 + y[i]**2))
    f['theta'].append(np.arctan(x[i]/y[i]))

    

data = DataFrame(f)
data.to_csv("//home//ban94gy//jupyter//ML//exer//END//momentum//ha_rt.csv")


In [29]:
###############################import###############################
import operator
import math
import random
import pandas as pd
import numpy
import time
import os
import dill
from sympy import *

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from scoop import futures
from pandas import DataFrame

from concurrent.futures import ProcessPoolExecutor
import matplotlib.pyplot as plt
import multiprocessing as mp

###################################################################

###################### Define functions############################
def safeDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1
    
def sqrt(left):
    if left>=0:
        return left**0.5
    else:
        return 0
    
def expe(left):
    if left>=0:
        return numpy.exp(-left)
    else:
        return 0




pset = gp.PrimitiveSet("MAIN", 7)
pset.addPrimitive(operator.add, 2)
#pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(expe, 1)
#set.addPrimitive(safeDiv, 2)
pset.addPrimitive(sqrt,1)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)

#pset.addEphemeralConstant("rand", lambda: random.randint(1,5))
#pset.addEphemeralConstant("rand"+str(ran1)+str(ran2), lambda: random.randint(min(ran1,ran2),max(ran1,ran2)))
pset.renameArguments(ARG0='t')
pset.renameArguments(ARG1='c1')
pset.renameArguments(ARG2='c2')
pset.renameArguments(ARG3='c3')
pset.renameArguments(ARG4='c4')
pset.renameArguments(ARG5='c5')
pset.renameArguments(ARG6='c6')

pool = mp.Pool()



###################################################################
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)



#########################input DATA#################################

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func1 = toolbox.compile(expr=individual)

    # Evaluate the mean squared error between the expression
    sqerrors = ((func1(t,c1,c2,c3,c4,c5,c6) - f1[t])**2 for t,c1,c2,c3,c4,c5,c6 in points)

    try:
        return max(sqerrors),
    except IndexError:
        return max(sqerrors),

g = pd.read_csv('ha_rt.csv')
p = g['r']

f1 = {}
for i in range(len(t)):
    f1[t[i]] = p[i]

toolbox.register("map", pool.map)

pts=[]

for i in range(len(t)):
    pts.append((t[i],1,2,3,4,5,0.5))

    
toolbox.register("evaluate", evalSymbReg, points=pts)
toolbox.register("select", tools.selTournament, tournsize=100)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=1, max_=3)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))


def main():
    pop = toolbox.population(n=5000)
    hof1 = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)
    
    
    #############################################C.O, Muta,Gener##########
    try:
        pop, log = algorithms.eaSimple(pop, toolbox, 0.8, 0.8, 500, stats=mstats,
                                       halloffame=hof1, verbose=True)
        print (hof1[0])

        return hof1
    
    except MemoryError:
        print (hof1[0])

        return hof1   
    
    except KeyboardInterrupt:
        print (hof1[0])

        return hof1   


if __name__ == "__main__":
    hof1 = main()
    

/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


   	      	    fitness     	    size   
   	      	----------------	-----------
gen	nevals	max    	min     	max	min
0  	5000  	88001.1	0.249999	14 	2  
1  	4799  	71660.5	0.0899999	20 	2  
2  	4803  	3.79782e+06	0.00743946	21 	2  
3  	4793  	5.3624e+06 	0.00249999	25 	2  
4  	4782  	558681     	0.00153691	33 	2  
5  	4767  	3.21181e+06	0.00142162	33 	2  
6  	4796  	669807     	0.00063033	37 	2  
7  	4786  	1.55492e+06	0.00063033	35 	2  
8  	4808  	159408     	0.00063033	34 	2  
9  	4808  	355502     	0.00063033	30 	2  
10 	4819  	3.96841e+08	0.00063033	34 	2  
11 	4790  	176049     	0.00063033	33 	2  
12 	4780  	2.39208e+07	0.00063033	44 	2  
13 	4788  	128802     	0.00063033	40 	2  
14 	4757  	996502     	0.00063033	35 	2  
15 	4797  	217986     	0.00063033	35 	2  
16 	4810  	1.9567e+07 	0.00063033	33 	2  
17 	4812  	1.21875e+06	0.00063033	31 	2  
18 	4805  	503629     	0.00063033	33 	2  
19 	4789  	1.34898e+06	0.00063033	33 	2  
20 	4787  	575677     	0.000628059	38 	2  
21 	4807  	7

Process ForkPoolWorker-972:
Process ForkPoolWorker-965:
Process ForkPoolWorker-980:
Process ForkPoolWorker-991:
Process ForkPoolWorker-999:
Process ForkPoolWorker-973:
Process ForkPoolWorker-974:
Process ForkPoolWorker-979:
Process ForkPoolWorker-971:
Process ForkPoolWorker-968:
Process ForkPoolWorker-984:
Process ForkPoolWorker-963:
Process ForkPoolWorker-964:
Process ForkPoolWorker-966:
Process ForkPoolWorker-983:
Process ForkPoolWorker-998:
Process ForkPoolWorker-976:
Process ForkPoolWorker-970:
Process ForkPoolWorker-961:
Process ForkPoolWorker-969:
Process ForkPoolWorker-962:
Traceback (most recent call last):
Process ForkPoolWorker-978:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process ForkPoolWorker-981:
Process ForkPoolWorker-967:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback 

  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/

  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/mul

In [30]:
pset = gp.PrimitiveSet("MAIN", 7)
pset.addPrimitive(operator.add, 2)
#pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(expe, 1)
#set.addPrimitive(safeDiv, 2)
pset.addPrimitive(sqrt,1)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)

#pset.addEphemeralConstant("rand", lambda: random.randint(1,5))
#pset.addEphemeralConstant("rand"+str(ran1)+str(ran2), lambda: random.randint(min(ran1,ran2),max(ran1,ran2)))
pset.renameArguments(ARG0='t')
pset.renameArguments(ARG1='c1')
pset.renameArguments(ARG2='c2')
pset.renameArguments(ARG3='c3')
pset.renameArguments(ARG4='c4')
pset.renameArguments(ARG5='c5')
pset.renameArguments(ARG6='c6')

pool = mp.Pool()



###################################################################
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)



#########################input DATA#################################

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func2 = toolbox.compile(expr=individual)

    # Evaluate the mean squared error between the expression
    sqerrors = ((func2(t,c1,c2,c3,c4,c5,c6) - f2[t])**2 for t,c1,c2,c3,c4,c5,c6 in points)

    try:
        return max(sqerrors),
    except IndexError:
        return max(sqerrors),

g = pd.read_csv('ha_rt.csv')
q = g['theta']

f2 = {}
for i in range(len(t)):
    f2[t[i]] = q[i]

toolbox.register("map", pool.map)

pts=[]

for i in range(len(t)):
    pts.append((t[i],1,2,3,4,5,0.5))

    
toolbox.register("evaluate", evalSymbReg, points=pts)
toolbox.register("select", tools.selTournament, tournsize=100)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=1, max_=3)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))


def main():
    pop = toolbox.population(n=5000)
    hof2 = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)
    
    
    #############################################C.O, Muta,Gener##########
    try:
        pop, log = algorithms.eaSimple(pop, toolbox, 0.8, 0.8, 500, stats=mstats,
                                       halloffame=hof2, verbose=True)
        print (hof2[0])

        return hof2
    
    except MemoryError:
        print (hof2[0])

        return hof2   
    
    except KeyboardInterrupt:
        print (hof2[0])

        return hof2    


if __name__ == "__main__":
    hof2 = main()
    

/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


   	      	    fitness     	    size   
   	      	----------------	-----------
gen	nevals	max   	min     	max	min
0  	5000  	272323	0.249999	14 	2  
1  	4811  	1.59262e+07	0.209491	20 	2  
2  	4793  	2.86943e+08	0.209491	19 	2  
3  	4787  	6.72112e+06	0.143712	18 	2  
4  	4789  	298334     	0.112008	21 	2  
5  	4819  	996004     	0.00908622	25 	2  
6  	4793  	1.60444e+06	0.00714429	35 	2  
7  	4823  	61563.8    	0.00714429	29 	2  
8  	4809  	21608.9    	0.00602491	28 	2  
9  	4794  	9418.61    	0.00450223	29 	2  
10 	4793  	299920     	0.00371176	31 	2  
11 	4785  	199630     	0.00371176	31 	2  
12 	4788  	5668.95    	0.00371176	33 	2  
13 	4782  	2227.79    	0.00371176	32 	2  
14 	4804  	12321      	0.00371176	32 	2  
15 	4820  	18708.9    	0.00371176	32 	2  
16 	4783  	81402      	0.00371176	32 	2  
17 	4777  	2828.11    	0.00371176	31 	2  
18 	4794  	169643     	0.00176842	30 	2  
19 	4808  	10009.9    	0.00176842	31 	2  
20 	4791  	48814.2    	0.00171894	32 	2  
21 	4795  	9692.66

Process ForkPoolWorker-1158:
Process ForkPoolWorker-1159:
Process ForkPoolWorker-1155:
Process ForkPoolWorker-1157:
Process ForkPoolWorker-1143:
Process ForkPoolWorker-1156:
Process ForkPoolWorker-1131:
Process ForkPoolWorker-1153:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, i

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-1140:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
   

  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.re

In [40]:
from sympy.parsing import sympy_parser


t = Symbol('t')


def add(a,b):
    return a+b

def expe(a):
    return exp(-a)

def mul(a,b):
    return a*b

def neg(a):
    return -a

def sqrt(a):
    return a**0.5


expr_dict = {'add':add, 'expe':expe, 'mul':mul,'neg':neg,'sqrt':sqrt, 'c1':1, 'c2':2, 'c3':3, 'c4':4,'c5':5, 'c6':0.5}

r = sympy_parser.parse_expr(str(hof1[0]), local_dict=expr_dict)
rr = diff(r,t)

theta = sympy_parser.parse_expr(str(hof2[0]), local_dict=expr_dict)
thethe = diff(theta,t)


r = lambdify(t,r)
rr = lambdify(t,rr)
theta = lambdify(t,theta)
thethe = lambdify(t,thethe)



In [46]:
t = np.arange(0.01,10,0.01)
r = r(t)
theta = theta(t)
dr = rr(t)
dtheta = thethe(t)

table = {'t': t, 'r' : r, 'theta' : theta, 'dr' : dr, 'dtheta' : dtheta}


data = DataFrame(table)
data.to_csv("//home//ban94gy//jupyter//ML//exer//END//momentum//ha_rt_d.csv")


TypeError: 'Series' object is not callable

In [5]:
import random
import numpy as np
import operator
import math
import pandas as pd
import time
import os
import dill
from sympy import *
from sympy.parsing import sympy_parser
from mpl_toolkits.mplot3d import Axes3D
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from scoop import futures
from pandas import DataFrame

from concurrent.futures import ProcessPoolExecutor
import matplotlib.pyplot as plt
import multiprocessing as mp

# def plot_pred(hof):
#     func = toolbox.compile(expr=hof[0])
#     y_hat = []
#     y = []
#     y_r = []
#     for a, i in enumerate(t):
#         y_hat.append(func(i))
#         y.append(p[a])
#         y_r.append(p_r[a])
        
#     from matplotlib import pyplot as plt
#     plt.plot(t,y)
#     plt.plot(t, y_hat)
#     plt.plot(t, y_r)
#     plt.xlabel('time')  # x-axis
#     plt.ylabel('x(t)')  # y-axis
#     plt.title('Data')  # title
#     plt.grid()  # grid
#     plt.legend(['with error','predicted','real'])
#     plt.show()  # plot show

###################### Define functions############################
def safeDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1
    
def sqrt(left):
    if left>=0:
        return left**0.5
    else:
        return 0
    
def expe(left):
    if left>=0:
        return np.exp(-left)
    else:
        return 0


pset = gp.PrimitiveSet("MAIN", 7)
pset.addPrimitive(operator.add, 2)
#pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(expe, 1)
#set.addPrimitive(safeDiv, 2)
pset.addPrimitive(sqrt,1)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)

pset.renameArguments(ARG0='r')
# pset.renameArguments(ARG1='theta')
pset.renameArguments(ARG2='c1')
pset.renameArguments(ARG3='c2')
pset.renameArguments(ARG4='c3')
pset.renameArguments(ARG5='c4')
pset.renameArguments(ARG6='c5')
pset.renameArguments(ARG7='c6')


###################################################################
pool = mp.Pool()

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
#toolbox.register("map", futures.map)

#########################input DATA#################################

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func3 = toolbox.compile(expr=individual)

    # Evaluate the mean squared error between the expression
    sqerrors = ((func3(r,c1,c2,c3,c4,c5,c6) - f3[t])**2 for t,r,c1,c2,c3,c4,c5,c6 in points)

    try:
        return max(sqerrors),
    except IndexError:
        return max(sqerrors),
    
toolbox.register("map", pool.map)



g = pd.read_csv('ha_rt_d.csv')
r = g['r']
theta = g['theta']
dr = g['dr']
dtheta = g['dtheta']

f3 = {}
for i in range(len(t)):
    f3[t[i]] = 0.5*dr[i]**2 + 0.5*r[i]**2*dtheta[i]**2
    
    
pts=[]

for i in range(len(t)):
    pts.append((t[i],r[i],1,2,3,4,5,0.5))
  
    

    
toolbox.register("evaluate", evalSymbReg, points=pts)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=1, max_=3)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))

def main():
    pop = toolbox.population(n=20000)
    hof3 = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    
    mstats.register("min", np.min)
    mstats.register("max", np.max)
    
    
    #############################################C.O, Muta,Gener##########
    try:
        pop, log = algorithms.eaSimple(pop, toolbox, 0.8, 0.8, 1000, stats=mstats,
                                       halloffame=hof3, verbose=True)
        print (hof3[0])

        return hof3
    
    except MemoryError:
        print (hof3[0])

        return hof3
    
    except KeyboardInterrupt:
        print (hof3[0])

        return hof3



if __name__ == "__main__":    
     hof3 = main()




/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


   	      	    fitness    	    size   
   	      	---------------	-----------
gen	nevals	max   	min    	max	min
0  	20000 	508622	2.78897	14 	2  
1  	19245 	518258	1.5416 	19 	2  
2  	19173 	4.66864e+06	0.942505	24 	2  
3  	19197 	30625      	0.94802 	25 	2  
4  	19249 	155280     	0.930634	31 	2  
5  	19195 	205903     	0.930634	32 	2  
6  	19196 	82910      	0.930634	31 	2  
7  	19219 	156581     	0.775947	31 	2  
8  	19198 	40688.3    	0.73082 	32 	2  
9  	19092 	275560     	0.400458	34 	2  
10 	19173 	245674     	0.295778	34 	2  
11 	19205 	40875.2    	0.209827	39 	2  
12 	19183 	187329     	0.192205	40 	2  
13 	19234 	40168.4    	0.142093	43 	2  
14 	19188 	3.51889e+06	0.0878859	43 	2  
15 	19240 	807980     	0.085113 	45 	2  
16 	19204 	6.32526e+06	0.0920141	47 	2  
17 	19197 	58485.6    	0.0651737	50 	2  
18 	19226 	232261     	0.0586564	48 	2  
19 	19224 	293634     	0.0373657	51 	2  
20 	19181 	247109     	0.0309623	49 	2  
21 	19227 	1.46026e+06	0.0309623	49 	2  
22 	19175 	2

Process ForkPoolWorker-232:


add(add(cos(add(add(mul(r, c3), mul(c2, c4)), mul(sin(c2), expe(r)))), cos(add(cos(mul(r, r)), mul(sin(c3), ARG1)))), add(sin(cos(add(mul(r, c3), mul(c2, c4)))), cos(neg(mul(cos(c4), mul(c3, c5))))))


Process ForkPoolWorker-239:
Process ForkPoolWorker-223:
Process ForkPoolWorker-231:
Process ForkPoolWorker-226:
Process ForkPoolWorker-238:
Process ForkPoolWorker-240:
Process ForkPoolWorker-237:
Process ForkPoolWorker-235:
Process ForkPoolWorker-233:
Process ForkPoolWorker-221:
Process ForkPoolWorker-224:
Process ForkPoolWorker-234:
Process ForkPoolWorker-262:
Process ForkPoolWorker-248:
Process ForkPoolWorker-272:
Process ForkPoolWorker-268:
Process ForkPoolWorker-228:
Process ForkPoolWorker-260:
Process ForkPoolWorker-236:
Process ForkPoolWorker-242:
Process ForkPoolWorker-274:
Traceback (most recent call last):
Process ForkPoolWorker-225:
Process ForkPoolWorker-230:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib

Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
KeyboardInterrupt
  File "/usr/l

Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-4-2c223f956e40>", line 109, in evalSymbReg
    return max(sqerrors),
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
Process ForkPoolWorker-246:
  File "/usr

  File "<ipython-input-4-2c223f956e40>", line 109, in evalSymbReg
    return max(sqerrors),
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__

  File "/usr/lib/python3.5/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-2c223f956e40>", line 106, in <genexpr>
    sqerrors = ((func3(r,c1,c2,c3,c4,c5,c6) - f3[t])**2 for t,r,c1,c2,c3,c4,c5,c6 in points)
  File "<ipython-input-4-2c223f956e40>", line 109, in evalSymbReg
    return max(sqerrors),
KeyboardInterrupt
  File "<ipython-input-4-2c223f956e40>", line 109, in evalSymbReg
    return max(sqerrors),
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<string>", line 1, in <lambda>
  File "<ipython-input-4-2c223f956e40>", line 106, in <genexpr>
    sqerrors = ((func3(r,c1,c2,c3,c4,c5,c6) - f3[t])**2 for t,r,c1,c2,c3,c4,c5,c6 in points)
  File "<ipython-input-4-2c223f956e40>", line 109, in evalSymbReg
    return max(sqerrors),
KeyboardInterrup

KeyboardInterrupt
  File "<ipython-input-4-2c223f956e40>", line 109, in evalSymbReg
    return max(sqerrors),
KeyboardInterrupt
  File "<ipython-input-4-2c223f956e40>", line 109, in evalSymbReg
    return max(sqerrors),
  File "<string>", line 1, in <lambda>
  File "<string>", line 1, in <lambda>
  File "<ipython-input-4-2c223f956e40>", line 60, in expe
    return np.exp(-left)
  File "<ipython-input-4-2c223f956e40>", line 109, in evalSymbReg
    return max(sqerrors),
  File "<ipython-input-4-2c223f956e40>", line 106, in <genexpr>
    sqerrors = ((func3(r,c1,c2,c3,c4,c5,c6) - f3[t])**2 for t,r,c1,c2,c3,c4,c5,c6 in points)
KeyboardInterrupt
  File "<ipython-input-4-2c223f956e40>", line 106, in <genexpr>
    sqerrors = ((func3(r,c1,c2,c3,c4,c5,c6) - f3[t])**2 for t,r,c1,c2,c3,c4,c5,c6 in points)
  File "<ipython-input-4-2c223f956e40>", line 106, in <genexpr>
    sqerrors = ((func3(r,c1,c2,c3,c4,c5,c6) - f3[t])**2 for t,r,c1,c2,c3,c4,c5,c6 in points)
  File "<ipython-input-4-2c223f956e40>

In [44]:
from sympy.parsing import sympy_parser


t = Symbol('t')


def add(a,b):
    return a+b

def expe(a):
    return exp(-a)

def mul(a,b):
    return a*b

def neg(a):
    return -a

def sqrt(a):
    return a**0.5


expr_dict = {'add':add, 'expe':expe, 'mul':mul,'neg':neg,'sqrt':sqrt, 'c1':1, 'c2':2, 'c3':3, 'c4':4,'c5':5, 'c6':0.5}

V = sympy_parser.parse_expr(str(hof3[0]), local_dict=expr_dict)
V





((theta + 4.87758256189037)**0.5 + sin(sin(2*theta)))*cos(5*theta) + cos(cos(5*theta)) + cos(cos(cos(theta))) + exp(-cos(sin(4))**0.5)